Required times per recipe for reference:
R1: 

peel 1 
cut 123-> 15 
cook 2 -> 30
fry 1  -> 10
mash 1  -> 6
combine 1 -> 2
assemble 1 -> 2
deliver 1 -> 0

total= 65

R2 
cut 123 -> 15
cook 12 -> 30
bake 1 -> 20
assemble 1 -> 2
combine 1  -> 2 
deliver -> 0

total = 69

total = 6700 when nothing is parallelized, and everything is available (1 robot all and no setup times)

In [2]:
import os 

files = os.listdir()
files.remove("analysis.ipynb")
results = dict()

for file in files: 
    results.update({file:dict()})
    results[file].update({"heuristic":file[0:4]})
    results[file].update({"toolDist":file[-10]})
    with open(file, "rt") as myfile: 
                results[file].update({"leadTime":file[-8:-4]})
                for line in myfile:  
                    if "Putting" in line: 
                        line = line.split("[")[1].split("]")[0]
                        [key,value] = line.split(",")
                        value = int(value.split("(")[1].split(")")[0])
                        results[file].update({key:value})


In [3]:
import pandas as pd 

resultDF = pd.DataFrame.from_dict(results).transpose()
resultDF = resultDF.apply(pd.to_numeric, errors="ignore")
resultDF.dtypes



heuristic         object
toolDist           int64
leadTime           int64
s6_notSwapped      int64
s2_swapped         int64
s3_notSwapped      int64
s1_workingTime     int64
s4_workingTime     int64
s1_notSwapped      int64
s4_swapped         int64
s6_swapped         int64
s4_notSwapped      int64
s3_workingTime     int64
s6_workingTime     int64
s5_notSwapped      int64
s2_workingTime     int64
s5_swapped         int64
s5_workingTime     int64
s2_notSwapped      int64
s1_swapped         int64
s3_swapped         int64
dtype: object

In [4]:

wTimes = ["s1_workingTime","s2_workingTime","s3_workingTime","s4_workingTime","s5_workingTime","s6_workingTime"]


for wTime in wTimes:
    resultDF["%"+wTime] = resultDF[wTime]/resultDF["leadTime"]
  #  print(resultDF[wTime])
  
resultDF

,heuristic,toolDist,leadTime,s6_notSwapped,s2_swapped,s3_notSwapped,s1_workingTime,s4_workingTime,s1_notSwapped,s4_swapped,...,s5_workingTime,s2_notSwapped,s1_swapped,s3_swapped,%s1_workingTime,%s2_workingTime,%s3_workingTime,%s4_workingTime,%s5_workingTime,%s6_workingTime
minTime_01_3237.txt,minT,1,3237,0,55,129,2444,1436,141,47,...,405,117,68,46,0.755020,0.596849,0.546185,0.443621,0.125116,0.001545
minTime_01_3428.txt,minT,1,3428,0,48,105,2555,1738,148,54,...,405,116,69,53,0.745333,0.474329,0.496208,0.507001,0.118145,0.001459
minTime_01_3700.txt,minT,1,3700,0,54,124,2457,1754,140,56,...,405,110,72,44,0.664054,0.464595,0.459459,0.474054,0.109459,0.001351
random_01_3268.txt,rand,1,3268,0,61,124,1590,1945,93,61,...,405,133,54,53,0.486536,0.645655,0.611995,0.595165,0.123929,0.001530
random_01_3590.txt,rand,1,3590,0,63,104,2131,1967,130,72,...,405,111,67,52,0.593593,0.537326,0.485515,0.547911,0.112813,0.001393
random_01_3162.txt,rand,1,3162,0,54,109,1961,2173,98,64,...,405,124,72,46,0.620177,0.623656,0.497786,0.687223,0.128083,0.001581
lastTool_01_4676.txt,last,1,4676,0,1,49,0,1835,0,4,...,405,298,0,1,0.000000,0.960222,0.043841,0.392429,0.086612,0.001069
lastTool_01_4544.txt,last,1,4544,0,4,49,1430,261,284,2,...,405,302,1,1,0.314701,1.000000,0.067121,0.057438,0.089129,0.001100
lastTool_01_4520.txt,last,1,4520,0,0,300,1780,230,342,2,...,405,0,2,2,0.393805,0.000000,1.000000,0.050885,0.089602,0.001106
minBooking_01_3228.txt,minB,1,3228,0,44,107,2275,1941,151,64,...,405,109,62,52,0.704771,0.521995,0.529430,0.601301,0.125465,0.001549


In [5]:
resultDF["avgLeadTime"] = resultDF.groupby(["heuristic","toolDist"])["leadTime"].transform("mean")
resultDF["utilization"] = resultDF[["%s1_workingTime","%s2_workingTime","%s3_workingTime","%s4_workingTime"]].mean(axis=1)
resultDF["avgUtilization"] = resultDF.groupby(["heuristic","toolDist"])["utilization"].transform("mean")
resultDF["\#setups"] = resultDF[["s1_swapped","s2_swapped","s3_swapped","s4_swapped"]].sum(axis=1)
resultDF["avgSetups"] = resultDF.groupby(["heuristic","toolDist"])["\#setups"].transform("mean")

In [6]:
resultDF= resultDF.drop(["s1_notSwapped", "s2_notSwapped", "s3_notSwapped", "s4_notSwapped","s5_swapped", "s5_notSwapped","s6_swapped", "s6_notSwapped","%s5_workingTime","%s6_workingTime"],axis=1)
resultDF= resultDF.drop(wTimes,axis=1)


In [10]:
resultDF = resultDF.reindex(sorted(resultDF.columns), axis=1)
resultDF = resultDF.sort_values(by=["toolDist","heuristic"])
resultDF

,%s1_workingTime,%s2_workingTime,%s3_workingTime,%s4_workingTime,\#setups,avgLeadTime,avgSetups,avgUtilization,heuristic,leadTime,s1_swapped,s2_swapped,s3_swapped,s4_swapped,toolDist,utilization
lastTool_01_4676.txt,0.000000,0.960222,0.043841,0.392429,6,4580.000000,6.666667,0.356704,last,4676,0,1,1,4,1,0.349123
lastTool_01_4544.txt,0.314701,1.000000,0.067121,0.057438,8,4580.000000,6.666667,0.356704,last,4544,1,4,1,2,1,0.359815
lastTool_01_4520.txt,0.393805,0.000000,1.000000,0.050885,6,4580.000000,6.666667,0.356704,last,4520,2,0,2,2,1,0.361173
minBooking_01_3228.txt,0.704771,0.521995,0.529430,0.601301,222,3225.000000,233.333333,0.594573,minB,3228,62,44,52,64,1,0.589374
minBooking_01_3146.txt,0.787349,0.527336,0.578512,0.559123,243,3225.000000,233.333333,0.594573,minB,3146,80,44,58,61,1,0.613080
minBooking_01_3301.txt,0.781278,0.499243,0.530748,0.513784,235,3225.000000,233.333333,0.594573,minB,3301,83,48,60,44,1,0.581263
minTime_01_3237.txt,0.755020,0.596849,0.546185,0.443621,216,3455.000000,222.000000,0.552226,minT,3237,68,55,46,47,1,0.585419
minTime_01_3428.txt,0.745333,0.474329,0.496208,0.507001,224,3455.000000,222.000000,0.552226,minT,3428,69,48,53,54,1,0.555718
minTime_01_3700.txt,0.664054,0.464595,0.459459,0.474054,226,3455.000000,222.000000,0.552226,minT,3700,72,54,44,56,1,0.515541
random_01_3268.txt,0.486536,0.645655,0.611995,0.595165,229,3340.000000,239.666667,0.577712,rand,3268,54,61,53,61,1,0.584838


In [11]:
resultDF.to_latex(index=False,formatters={"name": str.upper}, float_format="{:.2f}".format)

'\\begin{tabular}{rrrrrrrrlrrrrrrr}\n\\toprule\n%s1_workingTime & %s2_workingTime & %s3_workingTime & %s4_workingTime & \\#setups & avgLeadTime & avgSetups & avgUtilization & heuristic & leadTime & s1_swapped & s2_swapped & s3_swapped & s4_swapped & toolDist & utilization \\\\\n\\midrule\n0.00 & 0.96 & 0.04 & 0.39 & 6 & 4580.00 & 6.67 & 0.36 & last & 4676 & 0 & 1 & 1 & 4 & 1 & 0.35 \\\\\n0.31 & 1.00 & 0.07 & 0.06 & 8 & 4580.00 & 6.67 & 0.36 & last & 4544 & 1 & 4 & 1 & 2 & 1 & 0.36 \\\\\n0.39 & 0.00 & 1.00 & 0.05 & 6 & 4580.00 & 6.67 & 0.36 & last & 4520 & 2 & 0 & 2 & 2 & 1 & 0.36 \\\\\n0.70 & 0.52 & 0.53 & 0.60 & 222 & 3225.00 & 233.33 & 0.59 & minB & 3228 & 62 & 44 & 52 & 64 & 1 & 0.59 \\\\\n0.79 & 0.53 & 0.58 & 0.56 & 243 & 3225.00 & 233.33 & 0.59 & minB & 3146 & 80 & 44 & 58 & 61 & 1 & 0.61 \\\\\n0.78 & 0.50 & 0.53 & 0.51 & 235 & 3225.00 & 233.33 & 0.59 & minB & 3301 & 83 & 48 & 60 & 44 & 1 & 0.58 \\\\\n0.76 & 0.60 & 0.55 & 0.44 & 216 & 3455.00 & 222.00 & 0.55 & minT & 3237 & 68 & 5

In [54]:

smallResults = resultDF[["heuristic", "toolDist", "avgLeadTime", "avgUtilization", "avgSetups"]]
smallResults = smallResults.drop_duplicates()
smallResults = smallResults.reindex(sorted(smallResults.columns), axis=1)
smallResults.sort_values(by=["heuristic","toolDist"])

smallResults

,avgLeadTime,avgSetups,avgUtilization,heuristic,toolDist
minTime_01_3237.txt,3455.000000,222.000000,0.552226,minT,1
random_01_3268.txt,3340.000000,239.666667,0.577712,rand,1
lastTool_01_4676.txt,4580.000000,6.666667,0.356704,last,1
minBooking_01_3228.txt,3225.000000,233.333333,0.594573,minB,1
minTime_02_4690.txt,4648.333333,4.000000,0.350702,minT,2
random_02_4690.txt,4705.000000,4.000000,0.346442,rand,2
minBooking_02_4665.txt,4663.333333,4.000000,0.349560,minB,2
lastTool_02_4695.txt,4686.666667,4.000000,0.347796,last,2
lastTool_03_4505.txt,4505.333333,4.666667,0.361979,last,3
minBooking_03_3594.txt,3573.000000,57.000000,0.478276,minB,3


In [55]:
smallResults.to_latex(index=False,formatters={"name": str.upper}, float_format="{:.2f}".format)

'\\begin{tabular}{rrrlr}\n\\toprule\navgLeadTime & avgSetups & avgUtilization & heuristic & toolDist \\\\\n\\midrule\n3455.00 & 222.00 & 0.55 & minT & 1 \\\\\n3340.00 & 239.67 & 0.58 & rand & 1 \\\\\n4580.00 & 6.67 & 0.36 & last & 1 \\\\\n3225.00 & 233.33 & 0.59 & minB & 1 \\\\\n4648.33 & 4.00 & 0.35 & minT & 2 \\\\\n4705.00 & 4.00 & 0.35 & rand & 2 \\\\\n4663.33 & 4.00 & 0.35 & minB & 2 \\\\\n4686.67 & 4.00 & 0.35 & last & 2 \\\\\n4505.33 & 4.67 & 0.36 & last & 3 \\\\\n3573.00 & 57.00 & 0.48 & minB & 3 \\\\\n3054.33 & 47.67 & 0.55 & minT & 3 \\\\\n3228.67 & 56.33 & 0.53 & rand & 3 \\\\\n\\bottomrule\n\\end{tabular}\n'